In [ ]:
import matplotlib.pyplot as plt ## plotting package
import pandas as pd ## tabel package
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
# Basis URL zonder offset
URL1 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1961-1970'
URL2 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1971-1980'
URL3 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1981-1990'
URL4 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_1991-2000'
URL5 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_2001-2010'
URL6 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_2011-2020'
URL7 = 'https://en.wikipedia.org/wiki/List_of_human_spaceflights,_2021-present'
URLS = [URL1, URL2, URL3, URL4, URL5, URL6, URL7]

In [ ]:
class Vlucht:
    id = 0           # index
    jaar = 0         # jaar van lancering
    massa = 0        # massa van capsule/payload
    vluchtnaam = ""  # naam van vlucht
    link = ""        # link naar vluchtpagina
    rocketlink = ""  # link naar launch vehicle pagina
    rocketmass = 0   # massa van launch vehicle
    rocketname = ""  # naam van launch vehicle



In [ ]:
def lijst_halen(URL, vluchtlijst, start_id=0):
    id_nr = start_id
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    tabel = soup.find('tbody')
    rijen = tabel.find_all('tr')
    
    vluchten = vluchtlijst
    for rij in rijen[1:]: 
        if rij.find('th'): # check of de rij een header heeft
#             print('Header:' + rij.find('th').text)
            
            id_nr += 1
            vlucht = vlucht_halen(rij, id_nr)
            print(str(id_nr) + ". Naam: " + vlucht.vluchtnaam + ". Jaar: " + str(vlucht.jaar))
            
            if vlucht.rocketmass:
                massastring = str(vlucht.rocketmass)
            else:
                massastring = "Niet gevonden"

            print("Raket: " + vlucht.rocketname + ". Massa: " + massastring)
            
            vluchten.append(vlucht)
            print("*" * 40)
        else:
            continue

    
    return vluchten, id_nr

In [ ]:
def vlucht_halen(rij, nr):
    vlucht = Vlucht()
    vlucht.id = nr
    
    cells = rij.find_all('td')
    
    # basisinformatie van lancering uit tabel halen
    datumnaam = cells[1].text.split()
    datum = " ".join(datumnaam[0:2])
    vlucht.datum = datum
    jaar  = datumnaam[2]
    vlucht.jaar = int(jaar)
    naam  = " ".join(datumnaam[3:])
    vlucht.vluchtnaam = naam
    
    # link uit tabel halen, databoxes van betreffende pagina uitlezen
    urls = cells[1].find_all('a', href=True)
    link = 'https://en.wikipedia.org/' + urls[-1].get('href') # laatste link uit lijst halen
    vlucht.link = link
    
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    tabel = soup.find('table', {'class' : 'infobox'})
    headers = tabel.find_all('th', {'class' : 'infobox-label'})
    datarijen = tabel.find_all('td', {'class' : 'infobox-data'})
    
    #index ophalen van launch mass (lm)
    lm_index = 0
    for i in range(len(headers)):
        if headers[i].text == "Launch mass":
            lm_index = i
            break
            
    # launch mass inlezen
    try:
        massa_txt = datarijen[lm_index].text
        massa = massa_txt.split()[0]
        massa = massa.replace(',', '')
        massa = float(massa)
    except:
        massa = None
    vlucht.massa = massa
    
    # link ophalen van raket
    rocket_index = 0
    for i in range(len(headers)):
        if headers[i].text == "Rocket":
            rocket_index = i
            break
    
    if rocket_index > 0:
        rocket_url = datarijen[rocket_index].find('a', href=True)
        link = 'https://en.wikipedia.org/' + rocket_url.get('href')
        naam = rocket_url.text
        
        
        # check of raket al aanwezig is in dictionary
        if naam in raketten.keys():
            massa = raketten[naam]  # zo ja, ophalen uit dictionary
        else:
            massa = raketmassa_ophalen(link) # zo nee, ophalen via scrapen
            raketten[naam] = massa  # ...en toevoegen aan dictionary
        
        # alle raketdata toevoegen aan Vlucht object
        vlucht.rocketlink = link
        vlucht.rocketname = naam
        vlucht.rocketmass = massa
    
    ## return
    return vlucht

In [ ]:
unit_conversion = {'kg' : 1,
                   'kilogram' : 1,
                   'kilograms' : 1,
                   'lb' : 0.454,
                   'pounds' : 0.454,
                   'pound' : 0.454,
                   't' : 1000,
                   'tonne' : 1000
                   }
    

def raketmassa_ophalen(URL):
    mass = None   #default waarde, wordt veranderd als deze wordt gevonden
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    tabel = soup.find('table', {'class' : 'infobox hproduct'})
    headers = tabel.find_all('th', {'class' : 'infobox-label'})
    datarijen = tabel.find_all('td', {'class' : 'infobox-data'})
    rijen = tabel.find_all('tr')
    
    # range bepalen van de elementen die onder de size header vallen
    size_header_index = 0
    capacity_header_index = 0
    
    for i in range(len(rijen)):
        header = rijen[i].find('th')
        if header: # check of de de rij een header is
            if header.text == "Size":
                size_header_index = i
            elif header.text.strip() == "Capacity":
                capacity_header_index = i
    
    if size_header_index and capacity_header_index:   # check of de size header en cap header zijn gevonden
        # index van raket massa ophalen 
        for i in range(size_header_index, capacity_header_index):
            header = rijen[i].find('th')
            content = rijen[i].find('td')
            
            # check of de betreffende rij een header is
            if header:
                if header.text == "Mass": # check of het de juiste header is
                    mass_line = content.text.split()
                    unit_index = 0
                    unit = None
                    for key in unit_conversion.keys():
                        try:
                            unit_index = mass_line.index(key)
                            unit = key
                            break #breaks out of for loop if key is found
                        except:
                            continue

                    if unit_index != 0:
                        mass = mass_line[unit_index - 1]
                        mass = mass.replace(',', '')
                        mass = int(mass) * unit_conversion[unit]

    return mass

In [ ]:
startmsg = "Starttijd: "
endmsg = "Eindtijd: "

def printtime(message):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(message + current_time)

In [ ]:
vluchten = []    # initialiseer nieuwe vluchtlijst
raketten = {}
id_huidig = 0    # initialiseer index

start = datetime.now()
printtime(startmsg)
print("X" * 40)

for u in URLS:
    vluchten, id_huidig = lijst_halen(u, vluchten, start_id=id_huidig)
    
    print("X" * 60)
    printtime(endmsg)
    minutes = (datetime.now() - start).seconds // 60    #floor division by 60
    seconds = (datetime.now() - start).seconds %  60    #modulo by 60 
    print("Tijd verstreken: " + str(minutes) + ' minuten, ' + str(seconds) + ' seconden.')
    print("X" * 60)

In [ ]:
lst = []
for v in vluchten:
    lst.append([v.id, v.vluchtnaam, v.jaar, v.massa, v.rocketmass, v.rocketname])

In [ ]:
df = pd.DataFrame(lst, columns=['id', 'vluchtnaam', 'jaar', 'capsule_mass', 'rocketmass', 'rocketname'])

In [ ]:
yrmax = df.groupby('jaar').rocketmass.max()
jaren = df.jaar.unique()

In [ ]:
df[df.rocketmass==df.rocketmass.max()]

In [ ]:
plt.bar(jaren, yrmax)

In [ ]:
plt.scatter(jaren, yrmax/1000)
plt.xlabel('jaar')
plt.ylabel('Gewicht in ton')